In [1]:
import csv

In [7]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [7]:
# 分行 以篇為單位時用不到
def split_line(data_list):
    sentences = []
    for info in data_list[1:]:
        sentences.append(info[3].split('\n'))
        sentences.append(info[4].split('\n'))
    sentences = flatten_list(sentences)
    return sentences

In [24]:
def get_articles_list(data_list):
    articles = []
    for info in data_list[1:]:  # csv 第一行是欄位名稱的資料，不需要留下
        articles.append(info[3]+info[4])
    return articles

In [29]:
def extract_ckip_word(ws_list, pos_list):
    pos_remove_list = ['WHITESPACE', 'Ca', 'DE', 'P', 'T', 'I']  # 要過濾掉的詞性
    result = []
    for i in range(len(ws_list)):
        temp_sentence = []
        for j in range(len(ws_list[i])):
            for remove_pos in pos_remove_list:
                if remove_pos in pos_list[i][j]:
                    # print(remove_pos)
                    is_remove_pos = True
                    break
                else:
                    is_remove_pos = False
            if is_remove_pos == False:
                temp_sentence.append(ws_list[i][j])
        if temp_sentence != []:
            result.append(temp_sentence)
    extract_result = []
    for sentence in result:
        temp_sentence = []
        for word in sentence:
            temp_sentence.append(word.replace(' ', ''))
        extract_result.append(temp_sentence)
    return extract_result


def save_data(data, name):
    data_dict = {'text': data}
    data_df = pd.DataFrame(data_dict)
    data_df.to_pickle(name+'.pkl')

In [21]:
import csv
import pandas as pd
file = open('info_data/material_information_2021.csv',
            encoding="utf-8")  # 2021所有重訊
reader = csv.reader(file)
data_list = list(reader)
file.close()

In [4]:
# 去標點符號、數字
from zhon.hanzi import punctuation
import string
punctuation_str = punctuation+string.punctuation
number = '0123456789'
for info in data_list:
    # 去標點符號
    for i in punctuation_str:
        info[3] = info[3].replace(i, ' ')  # 3標題
        info[4] = info[4].replace(i, ' ')  # 4內文
    # 去數字
    for i in number:
        info[3] = info[3].replace(i, '')
        info[4] = info[4].replace(i, '')
    info[3] = info[3].replace('\t', '')
    info[3] = info[3].replace(' ', '')
    info[3] = info[3].replace('\n', ' ')
    info[4] = info[4].replace('\t', '')
    info[4] = info[4].replace(' ', '')
    info[4] = info[4].replace('\n', ' ')
# print(data_list[1][4])

In [28]:
articles = get_articles_list(data_list)
# print(articles[:2])

In [30]:
from ckiptagger import WS, POS, NER
ws_list = []
pos_list = []
ws = WS("./data")
pos = POS("./data")
count = 0
length = len(articles)
for text in articles:
    ws_results = ws([text])
    pos_results = pos(ws_results)
    ws_list = ws_list+ws_results
    pos_list = pos_list+pos_results
    count += 1
    if count % 300 == 0:  # 用來查看進度
        print(count, '/', length)
    if count % 5000 == 1000:  # 每5000篇會自動儲存
        temp_result = extract_ckip_word(ws_list, pos_list)  # 過濾詞性
        save_data(temp_result, 'FPM_input'+str(count))  # 儲存結果

In [31]:
# 全部完成後儲存
result = extract_ckip_word(ws_list, pos_list)
save_data(result, 'FPM_full_input')
# w2v_full_input的資料原本是有經過分行的(程式碼需加入分行split_line(data_list)，清除符號時不會去除\n)，這裡直接使用沒有分行的也可以
save_data(result, 'w2v_full_input')